# [Программирование на Python (SCS)](https://compscicenter.ru/courses/python/2015-autumn/classes/)

## Лектор Сергей Лебедев:  sergei.a.lebedev@gmail.com


|     **Дата**     |   **Название**  |     |
|:----------------:|:---------------:|:-----------------:|
| 26 октября 2015      |    Итераторы, генераторы itertools| 

# 8. Итераторы, генераторы, itertools

Напоминание: оператор **for** в Python работает с любой
последовательностью.

In [124]:
import dis
dis.dis("for x in xs: do_something(name)")

  1           0 SETUP_LOOP              20 (to 22)
              2 LOAD_NAME                0 (xs)
              4 GET_ITER
        >>    6 FOR_ITER                12 (to 20)
              8 STORE_NAME               1 (x)
             10 LOAD_NAME                2 (do_something)
             12 LOAD_NAME                3 (name)
             14 CALL_FUNCTION            1
             16 POP_TOP
             18 JUMP_ABSOLUTE            6
        >>   20 POP_BLOCK
        >>   22 LOAD_CONST               0 (None)
             24 RETURN_VALUE


Инструкция **GET_ITER** вызывает у аргумента оператора **for**
метод `__iter__`, который возвращает итератор.
- Инструкция FOR_ITER вызывает метод `__next__` у
итератора до тех пор, пока не будет поднято исключение <font color=red>**StopIteration**</font>.

# Слайд 2. Протокол итераторов

 Протокол итераторов состоит из двух методов:
- Метод `__iter__` возвращает экземпляр класса,
реализующего протокол итераторов, например, **self**.
- Метод `__next__` возвращает следующий по порядку
элемент итератора. Если такого элемента нет, то метод
должен поднять исключение <font color=red>**StopIteration**</font>.


 Важный инвариант метода `__next__`: если метод поднял
исключение <font color=red>**StopIteration**</font>, то все последующие вызовы
метода `__next__` тоже должны поднимать исключение.

 В отличие от, например, Java в Python **iterator** также
является **iterable**.

**Iterable** - то, что реализует iter()
```
Iterable is an object, which one can iterate over. It generates an Iterator when passed to iter() method. 
```

**Iterator** - то, что релазует iter() и next()
```
Iterator is an object, which is used to iterate over an iterable object using __next__() method. Iterators have __next__() method, which returns the next item of the object.

Note that every iterator is also an iterable, but not every iterable is an iterator. For example, a list is iterable but a list is not an iterator. An iterator can be created from an iterable by using the function iter(). To make this possible, the class of an object needs either a method __iter__, which returns an iterator, or a __getitem__ method with sequential indexes starting with 0.
```

# Слайд 3. Коллекции и итераторы

Для коллекций обычно нет смысла реализовывать
протокол итераторов целиком, достаточно реализовать
только метод `__iter__`.

 Иногда элементы коллекции можно перечислить более
чем одним способом. В этом случае удобно реализовывать
дополнительные методы, возвращающие итераторы

In [ ]:
class BinaryTree:
    def __iter__(self):
        return self.inorder_iter()
    def preorder_iter(self):
        # ...
    def inorder_iter(self):
        return InOrderIterator(self)
    def postorder_iter(self):
        # ...

# Слайд 4. Функции iter и next
 У функции `iter` две формы вызова:
- принимает итератор и вызывает у него метод `__iter__`,
- принимает функцию и терминальное значение и вызывает
функцию до тех пор, пока она не вернёт нужное значение

http://bit.ly/beautiful-python


In [ ]:
# читаем файл до конца порциями и обрабатываем каждую часть
# на помощь приходят Partial и iter
from functools import partial
    with open(path, "rb") as handle:
        read_block = partial(handle.read, 64)
        for block in iter(read_block, ""):
            do_something(block)

Функция next принимает итератор и вызывает у него
метод __next__. Можно также указать значение, которое
нужно вернуть в случае возникновения исключения
<font color=red>**StopIteration**</font>

In [125]:
next(iter([1, 2, 3]))

1

In [126]:
next(iter([]), 42)

42

# Слайд 5. “Семантика” оператора for

Напоминание:

In [ ]:
for x in xs:
    do_something(x)

Процесс исполнения оператора `for` можно концептуально
записать так:

In [ ]:
it = iter(xs)
while True:
    try:
        x = next(it)
    except StopIteration:
        break
    do_something(x)

# Слайд 6. Протокол итераторов и операторы in и not in
Операторы `in` и `not in` используют “магический” метод
`__contains__`, который возвращает **True**, если переданный
элемент содержится в экземпляре класса.
- По умолчанию метод `__contains__` реализован через
протокол итераторов:


In [ ]:
class object:
    # ...
    def __contains__(self, target):
        for item in self:
            if item == target:
                return True
        return False

Пример:

In [ ]:
id = Identity()  # -реализован на следующем слайде
5 in id          # ≡ id.__contains__(5)
42 not in id     # ≡ not id.__contains__(42)

# Слайд 7. Протокол итераторов и реализация “по умолчанию”
В Python предусмотрен упрощённый вариант реализации
протокола итераторов с использованием метода
`__getitem__`.

 Метод `__getitem__` принимает один аргумент — индекс
элемента в последовательности и:
- либо возвращает элемент, соответствующий индексу,
- либо поднимает **IndexError**, если элемента с таким
индексом нет.

После каждого обращения `__getitem__` увеличивает idx на 1, генерируя тем самым итератор по умолчанию:

In [127]:
class Identity:
    def __getitem__(self, idx):
        if idx > 5:
            raise IndexError(idx)
        return idx
list(Identity())


[0, 1, 2, 3, 4, 5]

# Слайд 8. “Семантика” упрощённого протокола итераторов: seq_iter


In [129]:
class seq_iter:
    def __init__(self, instance):
        self.instance = instance
        self.idx = 0
    def __iter__(self):
        return self
    def __next__(self):
        try:
            res = self.instance[self.idx]
        except IndexError:
            raise StopIteration
        self.idx += 1
        return res

# Слайд  9. “Семантика” упрощённого протокола итераторов: object


In [ ]:
class object:
# ...
    def __iter__(self):
        if not hasattr(self, "__getitem__"):
            cls = self.__class__
            msg = "{} object is not iterable"
            raise TypeError(msg.format(cls.__name__))
        return seq_iter(self)
    

# Слайд 10. Резюме: итераторы
В Python `iterator` также является `iterable`.
- Итератор — это экземпляр класса, который реализует два
метода `__init__` и `__next__`.
- Альтернативно можно воспользоваться реализацией этих
методов по умолчанию и определить метод `__getitem__`.
- Протокол итераторов используется:
>- оператором **for**,
>- операторами **in** и **not in**.
- Протокол итераторов реализуется всеми встроенными
коллекциями, а также, например, файлами и объектами
типа **map**, **filter** и **zip**.

# Слайд 11. Что такое генератор?
http://python.org/dev/peps/pep-0255

Генератор — это функция, которая использует не только
оператор **return**, но и оператор **yield**.
- В результате выполнения оператора **yield** работа функции
приостанавливается, а не прерывается, как при
использовании оператора **return**.


In [130]:
def g():
    print("Started")
    x = 42
    yield x
    x += 1
    yield x
    print("Done")

In [131]:
type(g)  # - g это функция

function

In [132]:
gen = g()
type(gen)  # вызов функции возвращает генератор!

generator

In [133]:
next(gen)  # next "проматывает" генератор до 1-го yield и возвращает значение х

Started


42

In [134]:
next(gen)  # - "промотка" до следующего yield 

43

In [135]:
next(gen)   # "промотка" до конца, который, как в итераторе вызывает 
            # StopIteration, если значения закончились

Done


StopIteration: 

# Слайд 12. Примеры генераторов: unique

In [139]:
def unique(iterable, seen=None):
    seen = set(seen or [])
    for item in iterable:
        if item not in seen:
            seen.add(item)
            yield item
xs = [1, 1, 2, 3, 4]
unique(xs)

<generator object unique at 0x05915060>

In [140]:
list(unique(xs))

[1, 2, 3, 4]

In [142]:
1 in unique(xs)

True

In [146]:
# Исчерпывание итератора свойственно также и генераторам:
def g():
    for i in range(5):
        print(i)
        yield i
gen = g()

2 in gen

0
1
2


True

In [147]:
list(gen)

3
4


[3, 4]

# Слайд 13. Примеры генераторов: map


In [148]:
def map(func, iterable, *rest):
#     zip склеит последовательности аргументов в кортежи аргументов
    for args in zip(iterable, *rest):
        yield func(*args)  # генерация результатов ф-ии по кортежам арг-ов
        
xs = range(5)
map(lambda x: x*x, xs)

<generator object map at 0x05915750>

In [149]:
list(map(lambda x: x * x, xs))


[0, 1, 4, 9, 16]

In [150]:
9 in map(lambda x: x * x, xs)

True

# Слайд 14. Примеры генераторов: chain
Конкатенация для людей, думающих о использовании памяти - перечисляет все итерабл:

In [151]:
def chain(*iterables):
    for iterable in iterables:
        for item in iterable:
            yield item
xs = range(3)
ys = [42]
chain(xs, ys)

<generator object chain at 0x05915510>

In [152]:
list(chain(xs, ys))

[0, 1, 2, 42]

In [153]:
42 in chain(xs, ys)

True

# Слайд 15. Примеры генераторов: count и enumerate

In [ ]:
def count(start=0):
    while True:
        yield start
        start += 1
next(count())

In [ ]:
counter = count()
next(counter)

In [ ]:
next(counter)

In [ ]:
def enumerate(iterable, start=0):
    pass  # как?


In [ ]:
next(enumerate(count(42)))


# Слайд 16. Переиспользование генераторов
Основное правило переиспользования генераторов: **не
делайте этого**.

In [ ]:
def g():
    yield 42
gen = g()
list(gen)

In [ ]:
list(gen)  # не тут-то было!


Если вы хотите переиспользовать генератор, подумайте
ещё раз.

Если вы уверены, что без переиспользования не обойтись,
воспользуйтесь функцией **tee** из модуля **itertools**.

# Слайд 17. Коллекции и генераторы
Генераторы позволяют компактно реализовывать метод
`__iter__` у коллекций

Рассмотрим уже знакомый нам класс бинарного дерева

In [ ]:
class BinaryTree:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left, self.right = left, right
    def __iter__(self): # inorder
        for node in self.left:
            yield node.value
        yield self.value
        for node in self.right:
            yield node.value

Плюс генераторов в том, что они позволяют обойтись без
лишних классов, например, **InOrderIterator**

# Слайд 18. Выражения-генераторы
Напоминание: в Python есть генераторы списков, множеств
и словарей.
- Выражения-генераторы работают аналогичным образом,
но не порождают коллекцию в процессе работы:

In [ ]:
gen = (x ** 2 for x in range(10**42) if x % 2 == 1)
gen

In [ ]:
next(gen)

In [ ]:
list(filter(lambda x: x % 2 == 1, (x ** 2 for x in range(10))))

Если выражение-генератор — единственный аргумент
функции, скобки можно опустить:

In [ ]:
sum(x ** 2 for x in range(10) if x % 2 == 1)

# Слайд 19. Выражение yield
Оператор **yield** можно использовать как выражение:

In [ ]:
def g():
    res = yield # точка входа 1
    print("Got {!r}".format(res))
    res = yield 42 # точка входа 2
    print("Got {!r}".format(res))
gen = g()
next(gen)  # "промотаем" до первого yield
next(gen)  # "промотаем" до первого yield

In [ ]:
next(gen)

На первый взгляд выражение **yield** выглядит бесполезно,
но первое впечатление обманчиво.

# Слайд 20. Интерфейс генераторов: send
Метод **send** возобновляет выполнение генератора и
“отправляет” свой аргумент в следующий **yield**:

In [ ]:
gen = g()
gen.send("foobar")

Чтобы инициализировать генератор нужно “отправить” ему
**None**. Функция **next** делает ровно это:

In [ ]:
gen = g()
next(gen)

Результатом метода send является следующее значение
генератора или исключение <font color=red>StopIteration</font>, если такого
исключения нет.

In [ ]:
gen = g()
gen.send(None)  # == next(gen)
gen.send("foobar")

# Слайд 21. Интерфейс генераторов: throw
Метод **throw** поднимает переданное исключение в месте,
где генератор приостановил исполнение и возвращает
следующее значение генератора


In [ ]:
def g():
    try:
        yield 42
    except Exception as e:
        yield e
gen = g()
next(gen)

In [ ]:
gen.throw(ValueError, "something is wrong")

In [ ]:
gen.throw(RuntimeError, "another error")

Если генератор не обработал брошенное в него
исключение, то выполнение генератора прекращается и
исключение передаётся наверх по стеку вызовов.

# Слайд 22. Интерфейс генераторов: close
Метод **close** поднимает специальное исключение
<font color=red>**GeneratorExit**</font> в месте, где генератор приостановил
исполнение:


In [ ]:
def g():
    try:
        yield 42
    finally:
        print("Done")
        
gen = g()
next(gen)

In [ ]:
gen.close()

Если всё хорошо, то метод **close** завершает работу
генератора и ничего не возвращает.

 Что может пойти не так? Генератор может обработать
исключение <font color=red>**GeneratorExit**</font> и поднять другое исключение.

# Слайд 23. Генераторы ∼ сопрограммы aka coroutines
http://dabeaz.com/coroutines

Сопрограмма — это программа, которая может иметь
больше одной точки входа, а также поддерживает
остановку и продолжение с сохранением состояния.

Звучит как определение генератора наоборот:

In [ ]:
def grep(pattern):
    print("Looking for {!r}".format(pattern))
    while True:
        line = yield
        if pattern in line:
            print(line)

In [ ]:
gen = grep("Gotcha!")

In [ ]:
next(gen)

In [ ]:
gen.send("This line doesn't have what we're looking for")
gen.send("This one does. Gotcha!")

# Слайд 24. Инициализация сопрограмм

http://python.org/dev/peps/pep-0380

Прежде, чем начать работать с сопрограммой, её нужно
инициализировать с помощью вызова функции next.
• Объявим декоратор coroutine, который скроет эту деталь
реализации

In [ ]:
import functools
def coroutine(g):
    @functools.wraps(g)
    def inner(*args, **kwargs):
        gen = g()
        next(gen)
        return gen
    return inner
grep = coroutine(grep)
gen = grep("Gotcha!")
gen.send("One more line for ya!")
# Зачем это всё нужно? Ответ в домашнем задании

# Слайд 26. Оператор yield from

Оператор **yield from** позволяет делегировать выполнение
другому генератору:

In [70]:
def chain(*iterables):
    for iterable in iterables:
        yield from iterable



Любые вызовы методов **send** и **throw** у родительского
генератора будут переданы вложенному генератору без
изменений.

# Слайд 27. Оператор return и исключение StopIteration
Кроме оператора **yield** в теле генератора можно
использовать оператор **return**.
- На человеческом языке использование **return** означает:
«У меня больше нет элементов, извини, возьми лучше вот
это.»

In [71]:
def g():
    yield 42
    return [] # держи!

In [72]:
gen = g()
next(gen)

42

In [73]:
next(gen)

StopIteration: []

# Слайд 28. return !≡ raise StopIteration
Несмотря на схожесть, использование оператора **return** в
генераторе не эквивалентно поднятию исключения
<font color=red>**StopIteration**</font>.

In [ ]:
def g():
    try:
        yield 42
        raise StopIteration([]) # !≡ return []
    except Exception as e:
        pass

# Слайд 29. Выражение yield from
Оператор yield from, как и оператор yield, можно
использовать в качестве выражения.
• При этом значением выражения yield from будет
значение атрибута value у поднятого вложенным
генератором исключения <font color=red>**StopIteration**</font>:

In [75]:
def f():
    yield 42
    return []

def g():
    res = yield from f()
    print("Got {!r}".format(res))

gen = g()
next(gen)

42

In [76]:
next(gen, None)

Got []


# Слайд 30. Менеджеры контекста и генераторы: мотивация
Протокол менеджеров контекста требует реализации двух
методов: `__enter__` и `__exit__`,
- Если мы хотим, чтобы у менеджера было какое-то
состояние, то мы вынуждены также добавить метод
`__init__`.

В итоге получаем:

In [77]:
class cd:
    def __init__(self, path):
        self.path = path
    def __enter__(self):
        self.saved_cwd = os.getcwd()
        os.chdir(self.path)
    def __exit__(self, *exc_info):
        os.chdir(self.saved_cwd)

# Слайд 31. Менеджеры контекста и генераторы: @contextmanager
Декоратор **contextmanager** из модуля **contextlib**
принимает генератор специального вида и строит по нему
менеджер контекста.

In [ ]:
from contextlib import contextmanager

@contextmanager
def cd(path): # __init__
    old_path = os.getcwd() # __enter__
    os.chdir(path)
    try:
        yield # ---------
    finally:
        os.chdir(old_path) # __exit__

Генераторы позволяют сократить количество
синтаксического шума при реализации менеджеров
контекста

# Слайд 32. Ещё один пример использования @contextmanager
Метод ``__enter__``, построенный декоратором `contextmanager`,
возвращает аргумент оператора `yield`:

In [ ]:
import tempfile
import shutil
@contextmanager
def tempdir(): # __init__
    outdir = tempfile.mkdtemp() # __enter__
    try:
        yield outdir # ---------
    finally:
        shutil.rmtree(outdir) # __exit__
    
with tempdir() as path:
    print(path) # ==> /tmp/tmpvfzsmvsv

# Слайд 33. Генераторы: резюме
Генератор в Python — это функция, которая использует
операторы ``yield`` или `yield from`.

 В мире Python генераторы вездесущи не менее, чем
любимые всеми декораторы.

 Мы поговорили о том, что генераторы можно использовать
- как итераторы,
- как сопрограммы,
- как легкие потоки,
- для компактной реализации менеджеров контекста.

# Слайд 34. Модуль itertools: islice
Функция `islice` обобщает понятие слайса на
произвольный итератор:

In [80]:
from itertools import islice
xs = range(10)
list(islice(xs, 3))  # ≡ xs[:3]

[0, 1, 2]

In [81]:
list(islice(xs, 3, None)) # ≡ xs[3:]

[3, 4, 5, 6, 7, 8, 9]

In [82]:
list(islice(xs, 3, 8, 2)) # ≡ xs[3:8:2]

[3, 5, 7]

Прелесть функций из модуля `itertools` в том, что с
помощью них легко выражаются самые разнообразные
операции над последовательностями.

<font color=blue> Вопрос</font><br>
Как будет выглядеть функция `drop`, “выкидывающая” префикс
длины `n` из переданного ей итератора?

# Слайд 35. Модуль itertools: бесконечные итераторы
ля удобства реализуем родственника функции `drop`:
функцию `take`, которая строит список из более, чем `n`
первых элементов переданного ей итератора.

In [85]:
def take(n, iterable):
    return list(islice(iterable, n))

In [91]:
list(take(3, range(10)))

[0, 1, 2]

In [89]:
# Названия бесконечных итераторов говорят сами за себя:
from itertools import count, cycle, repeat
take(3, count(0, 5))

[0, 5, 10]

In [90]:
take(3, cycle([1, 2, 3]))

[1, 2, 3]

In [94]:
take(3, repeat(42))

[42, 42, 42]

In [95]:
take(3, repeat(42, 2)) # не совсем бесконечность

[42, 42]

# Слайд 36. Модуль itertools: dropwhile и takewhile
Функции `dropwhile` и `takewhile` обобщают логику функций
`drop` и `take` на произвольный предикат.

Обратите внимание, что обе функции возвращают
**итератор**, а не список, как реализованная нами функция
take:

In [97]:

from itertools import dropwhile, takewhile
list(dropwhile(lambda x: x < 5, range(10)))

[5, 6, 7, 8, 9]

In [99]:
it = takewhile(lambda x: x < 5, range(10))
it

In [100]:
list(it)

[0, 1, 2, 3, 4]

# Слайд 37. Модуль itertools: chain
В модуле `itertools` реализован уже знакомый нам
генератор `chain`, который конкатенирует произвольное
число итераторов:

In [102]:
from itertools import chain
take(5, chain(range(2), range(5, 10)))

[0, 1, 5, 6, 7]

Сконкатенировать итератор итераторов (!) можно с
помощью метода `chain.from_iterable`:

In [104]:
it = (range(x, x ** x) for x in range(2, 4))
take(5, chain.from_iterable(it))


[2, 3, 3, 4, 5]

<font color=blue>Вопрос</font><br>
Чем `chain.from_iterable(it)` отличается от `chain(*it)`?

# Слайд 38. Модуль itertools: tee
Функция `tee` создаёт `n` независимых копий переданного ей
итератора:


In [106]:
from itertools import tee
it = range(3)
a, b, c = tee(it, 3)
list(a), list(b), list(c)

([0, 1, 2], [0, 1, 2], [0, 1, 2])

Использовать it после копирования не рекомендуется,
потому что в этом случае скопированные итераторы
a, b, c могут пропустить элемент:
    

In [110]:
it = iter(range(3))
a, b = tee(it, 2)
used = list(it)
list(a), list(b)

([], [])

<font color=blue>Вопрос</font><br>
Что изменится, если убрать вызов функции iter из второго
примера?

# Слайд 39. Модуль itertools: комбинаторные итераторы
В модуле `itertools` в виде итераторов реализованы полезные
комбинаторные операции, например:

In [114]:
import itertools
# декартово произведение итераторов,
list(itertools.product("AB", repeat=2))

[('A', 'A'), ('A', 'B'), ('B', 'A'), ('B', 'B')]

In [115]:
list(itertools.product("AB", repeat=3))

[('A', 'A', 'A'),
 ('A', 'A', 'B'),
 ('A', 'B', 'A'),
 ('A', 'B', 'B'),
 ('B', 'A', 'A'),
 ('B', 'A', 'B'),
 ('B', 'B', 'A'),
 ('B', 'B', 'B')]

In [117]:
# перестановки элементов итератора,
list(itertools.permutations("AB"))

[('A', 'B'), ('B', 'A')]

In [121]:
# сочетания (с повторениями и без) из элементов итератора.
from itertools import combinations, \
    combinations_with_replacement
list(combinations("ABC", 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [122]:
list(combinations_with_replacement("ABC", 2))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

# Слайд 40. Комбинаторные итераторы и функция build_graph


In [123]:
def build_graph(words, mismatch_percent):
    g = ...
    n_words = len(words)
    for u, v in itertools.combinations(range(n_words), 2):
        if len(words[u]) != len(words[v]):
            continue
        distance = hamming(words[u], words[v])
        # ...
    return g

<font color=blue>Вопрос</font><br>
Выглядит неплохо, но можно лучше. Как?

# Слайд 41. Модуль itertools: резюме
Модуль `itertools` предоставляет обширный набор
компонент для реализации операций над
последовательностями.
 Мы обсудили:
- `islice`,
- бесконечные итераторы `count`, `cycle`, `repeat`,
- `chain`,
- `tee`,
- комбинаторные итераторы `product`, ``permutations``,
`combinations` и `combinations_with_replacement`.